In [ ]:
import numpy as np

import torchvision.models as models
import torch, time, datetime, cv2, os, random, torch, tqdm
import torch.nn as nn
from torch.utils.data import Dataset

print(os.getcwd())

/home/moooooo16/moooooo16/Python_Projects/Plant_Seedlings_Classification


In [ ]:
class DataGenerator(Dataset):
    def __init__(self, imgPath, target_size=(224, 224), shuffle=True, valid=False):
        self.imgPath_list, self.classes = [], []

        for i in sorted(os.listdir(imgPath), key=lambda x: int(x)):
            base_path = os.path.join(imgPath, i)
            for j in os.listdir(base_path):
                self.imgPath_list.append(os.path.join(base_path, j))
                self.classes.append(int(i))
        self.classes = self.classes
        self.imgPath_list, self.classes = np.array(self.imgPath_list), np.array(self.classes)
        self.target_size = target_size
        self.shuffle = shuffle
        self.valid = valid
        self.indexes = np.arange(len(self.imgPath_list))
        np.random.shuffle(self.indexes)

    def __len__(self):
        return len(self.imgPath_list)


    def __getitem__(self, index):
        if index == 0:
            np.random.shuffle(self.indexes)

        indexes = self.indexes[index]

        x, y = self.__data_generation(self.imgPath_list[indexes]), self.classes[indexes]
        x = np.array(x, dtype=np.float) / 255.0
        x = np.transpose(x, (2, 0, 1))

        return torch.from_numpy(x), y

    def __data_generation(self, img_path):
        random_int = random.randint(1, 7)
        if self.valid:
            random_int = 5

        img = cv2.imdecode(np.fromfile(img_path, np.uint8), cv2.IMREAD_COLOR)
        if random_int == 1:
            img = self.adjust_contrast_bright(img)
        elif random_int == 2:
            img = self.flip(img)
        elif random_int == 3:
            img = self.random_noise(img)
        elif random_int == 4:
            img = self.rotate(img)
        else:
            pass

        img = cv2.resize(img, (224, 224))
        return img

    def flip(self, img):

        mode = np.random.choice([-1, 0, 1])
        return cv2.flip(img, flipCode=mode)

    def random_noise(self, img, rand_range=(3, 20)):

        img = np.asarray(img, np.float)
        sigma = random.randint(*rand_range)
        nosie = np.random.normal(0, sigma, size=img.shape)
        img += nosie
        img = np.uint8(np.clip(img, 0, 255))
        return img

    def rotate(self, img, angle=30, scale=1.0):

        height, width = img.shape[:2]  # 高和宽
        center = (width / 2, height / 2)  # 中点
        angle = np.random.choice(np.arange(-angle, angle))

        M = cv2.getRotationMatrix2D(center, angle, scale)  # 绕着某一点的旋转
        rotated = cv2.warpAffine(img, M, (height, width))
        return rotated

    def adjust_contrast_bright(self, img, contrast=1.2, brightness=100):

        return np.uint8(np.clip((contrast * img + brightness), 0, 255))

In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=512, out_features=12, bias=True),)

In [ ]:
BATCH_SIZE = 64
DEVICE = torch.device("cuda")
model = model.to(DEVICE)

In [ ]:
train_generator = DataGenerator('train')
train_generator = torch.utils.data.DataLoader(train_generator, BATCH_SIZE, shuffle=True, num_workers=4)
valid_generator = DataGenerator('val', valid=True)
valid_generator = torch.utils.data.DataLoader(valid_generator, BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001, weight_decay=0.001)
loss = torch.nn.CrossEntropyLoss().to(DEVICE)

In [ ]:
if __name__ == '__main__':
    best_acc = 0
    print('{} begin train on {}!'.format(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), DEVICE))
    with open('train.log', 'w+') as f:
        f.write('loss,val_loss,acc,val_acc')

    for epoch in range(30):
        model.to(DEVICE)
        model.train()
        train_loss = 0
        correct = 0
        begin = time.time()
        for x, y in tqdm.tqdm(train_generator):
            x, y = x.to(DEVICE), y.to(DEVICE).long()

            pred = model(x.float())
            l = loss(pred, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

            y_pred = torch.max(pred.data, 1)
            correct += (y_pred.indices == y).sum().to(DEVICE).item()

            train_loss += float(l.data)
        train_loss /= len(train_generator)
        train_acc = (correct / (len(train_generator) * BATCH_SIZE))

        test_loss = 0
        correct = 0
        model.eval()
        with torch.no_grad(): #没有梯度
            for x, y in tqdm.tqdm(valid_generator):
                x, y = x.to(DEVICE), y.to(DEVICE).long()

                pred = model(x.float())
                l = loss(pred, y)
                test_loss += float(l.data)

                y_pred = torch.max(pred.data, 1)
                correct += (y_pred.indices == y).sum().to(DEVICE).item()

        test_loss /= len(valid_generator)
        test_acc = (correct / (len(valid_generator) * BATCH_SIZE))

        if test_acc > best_acc:
            best_acc = test_acc
            model.to(DEVICE)
            torch.save(model, 'model.pht')
        with open('train.log', 'a+') as f:
            f.write('\n{:.5f},{:.5f},{:.4f},{:.4f}'.format(train_loss, test_loss, train_acc, test_acc))
        print('{} epoch:{}, time:{:.2f}s, train_loss:{:.5f}, val_loss:{:.5f}, train_acc:{:.4f}, val_acc:{:.4f}'.format(
            datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            epoch + 1, time.time() - begin, train_loss, test_loss, train_acc, test_acc
        ))

2022-02-18 16:44:57 begin train on cuda!


  0%|          | 0/71 [00:00<?, ?it/s]/home/moooooo16/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/moooooo16/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/moooooo16/miniconda3

2022-02-18 16:45:13 epoch:1, time:15.92s, train_loss:0.36847, val_loss:0.28114, train_acc:0.8856, val_acc:0.9087


  0%|          | 0/71 [00:00<?, ?it/s]/home/moooooo16/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/moooooo16/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/moooooo16/miniconda3

2022-02-18 16:45:28 epoch:2, time:14.98s, train_loss:0.21056, val_loss:0.24670, train_acc:0.9322, val_acc:0.9167


...

2022-02-18 16:51:35 epoch:28, time:13.45s, train_loss:0.03615, val_loss:0.29387, train_acc:0.9806, val_acc:0.9171


  0%|          | 0/71 [00:00<?, ?it/s]/home/moooooo16/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/moooooo16/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/moooooo16/miniconda3

2022-02-18 16:51:49 epoch:29, time:13.79s, train_loss:0.04474, val_loss:0.26694, train_acc:0.9784, val_acc:0.9251


  0%|          | 0/71 [00:00<?, ?it/s]/home/moooooo16/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/moooooo16/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/moooooo16/miniconda3

2022-02-18 16:52:02 epoch:30, time:13.59s, train_loss:0.05380, val_loss:0.24550, train_acc:0.9745, val_acc:0.9351
